In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from sklearn.preprocessing import StandardScaler,OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve, roc_curve

In [13]:
default=pd.read_csv('/content/drive/MyDrive/Logestic_Regression/default.csv')

In [14]:
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [15]:
default.shape

(10000, 4)

In [16]:
default.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [17]:

default.isnull().sum()

default    0
student    0
balance    0
income     0
dtype: int64

No missing data sets

In [18]:
default['default'].unique()

array(['No', 'Yes'], dtype=object)

In [19]:
default['default'].value_counts()

No     9667
Yes     333
Name: default, dtype: int64

The classes have a different data pts,the data sets are imbalanced. The accuracy score is not helpfull to determine the performance of the model.

In [34]:
default['default'] = default['default'].replace(' ', np.nan).astype('float')

ValueError: ignored

In [20]:
default.describe()

,balance,income
count,10000.000000,10000.000000
mean,835.374886,33516.981876
std,483.714985,13336.639563
min,0.000000,771.967729
25%,481.731105,21340.462903
50%,823.636973,34552.644802
75%,1166.308386,43807.729272
max,2654.322576,73554.233495


### Prepare the data

In [21]:
default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [24]:
X=default.drop('student',axis=1)
y=default['default'].map({'No':0,'Yes':1})

In [25]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
# define categorical and numeric transformers
num_tran=Pipeline(
    steps=[('KNNImputer',KNNImputer(n_neighbors=5,weights='uniform',metric='p2')),
           ('scaler',StandardScaler())]
)

cat_tran=Pipeline(
    steps=[('simpleimputer', SimpleImputer(strategy='constant',fill_value='missing')),
           ('transformer',OneHotEncoder(handle_unknown='ignore'))]
)

In [27]:
# dispatch object columns to the categorical_transformer and remaining columns to numerical_transformer
preprocessor=ColumnTransformer(transformers=[('numerical',num_tran,make_column_selector(dtype_include='object')),
                                             ('categorical',cat_tran, make_column_selector(dtype_include='object'))])

In [28]:
#create a feature extractor or feature selector
extractor=SelectFromModel(estimator=LogisticRegression(penalty='l1',C=1,solver='liblinear',random_state=42,max_iter=1000))

In [30]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
lr_pipe=Pipeline(
    steps=[('transformer',preprocessor),
           ('selector',extractor),
           ('classifier',LogisticRegression(random_state=42,max_iter=1000))]
)
lr_pipe.fit(X_train,y_train)

ValueError: ignored